### Comparison of solution methods for order planning in maintenance with Google OR-Tools

In [16]:
import os
import csv
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

#### List of solution methods and data files

In [17]:
first_solution_strategies = [routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC, routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC,
                            routing_enums_pb2.FirstSolutionStrategy.PATH_MOST_CONSTRAINED_ARC, routing_enums_pb2.FirstSolutionStrategy.PATH_MOST_CONSTRAINED_ARC,
                            routing_enums_pb2.FirstSolutionStrategy.EVALUATOR_STRATEGY, routing_enums_pb2.FirstSolutionStrategy.SAVINGS,
                            routing_enums_pb2.FirstSolutionStrategy.SWEEP, routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES,
                            routing_enums_pb2.FirstSolutionStrategy.ALL_UNPERFORMED, routing_enums_pb2.FirstSolutionStrategy.BEST_INSERTION,
                            routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION, routing_enums_pb2.FirstSolutionStrategy.SEQUENTIAL_CHEAPEST_INSERTION,
                            routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION, routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC,
                            routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_ARC, routing_enums_pb2.FirstSolutionStrategy.FIRST_UNBOUND_MIN_VALUE]

local_search_metaheuristics = [routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC, routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT,
                              routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH, routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING,
                              routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH, routing_enums_pb2.LocalSearchMetaheuristic.GENERIC_TABU_SEARCH]


data_files = ['homberger_200_C1_2_1-time_matrix.csv', 'homberger_200_R1_2_1-time_matrix.csv', 'homberger_200_RC1_2_1-time_matrix.csv',  
              'solomon-100-c101-time_matrix.csv', 'solomon-100-r101-time_matrix.csv', 'solomon-100-rc101-time_matrix.csv',
              'solomon-25-c101-time_matrix.csv', 'solomon-25-r101-time_matrix.csv', 'solomon-25-rc101-time_matrix.csv', 
              'solomon-50-c101-time_matrix.csv', 'solomon-50-r101-time_matrix.csv', 'solomon-50-rc101-time_matrix.csv']

#### Get Data

In [18]:
script_dir = os.path.abspath('')
folder_data_for_comparison = os.path.join(script_dir, "data\data_for_comparison")
folder_results= os.path.join(script_dir, "results")

#### Functions

In [19]:
def create_data_model():
    data = {}
    data['time_matrix'] = time_matrix
    data['service_time'] = service_times
    data['num_vehicles'] = int(len(service_times) / 3)
    data['depot'] = 0
    return data

def print_solution(manager, routing, solution):
    #print(f'Objective: {solution.ObjectiveValue()}')
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(manager.GetNumberOfVehicles()):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += f'{manager.IndexToNode(index)} '
            plan_output += f'Time({solution.Value(time_var)}) -> '
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += f'{manager.IndexToNode(index)} '
        plan_output += f'Time({solution.Value(time_var)})\n'
        plan_output += f'Time of the route: {solution.Value(time_var)}min\n'
        #print(plan_output)
        total_time += solution.Value(time_var)
    #print(f'Total time of all routes: {total_time}min')
    return total_time

#### Main

In [20]:
for data_file in data_files:
    print(data_file)

    with open(os.path.join(folder_data_for_comparison, data_file), 'r') as read_obj:
        csv_reader = csv.reader(read_obj)
        time_matrix = []
        for line in csv_reader:
            time_matrix.append([int(x) for x in line])
    
    with open(os.path.join(folder_data_for_comparison, 'service_times.csv'), 'r') as read_obj:
        csv_reader = csv.reader(read_obj)
        service_times = []
        for line in csv_reader:
            service_times.append([int(x) for x in line])
        service_times = service_times[0][:len(time_matrix)]

    evaluation = []
    for i in range(len(first_solution_strategies)):
        for j in range(len(local_search_metaheuristics)):
            data = create_data_model()

            manager = pywrapcp.RoutingIndexManager(
                    len(data['time_matrix']),
                    data['num_vehicles'],
                    data['depot'])

            routing = pywrapcp.RoutingModel(manager)

            #Returns the travel time + service time between the two nodes.
            def time_callback(from_index, to_index):
                from_node = manager.IndexToNode(from_index)
                to_node = manager.IndexToNode(to_index)
                return data['time_matrix'][from_node][to_node] + data['service_time'][from_node]

            transit_callback_index = routing.RegisterTransitCallback(time_callback)
            routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

            time = 'Time'
            working_time = 420 # 7 hours = 420 min
            routing.AddDimension(transit_callback_index, 0, working_time, True, time)
            time_dimension = routing.GetDimensionOrDie(time)
            time_dimension.SetGlobalSpanCostCoefficient(10)

            search_parameters = pywrapcp.DefaultRoutingSearchParameters()
            search_parameters.first_solution_strategy = first_solution_strategies[i]
            search_parameters.local_search_metaheuristic = local_search_metaheuristics[j]
            search_parameters.time_limit.FromSeconds(10)

            solution = routing.SolveWithParameters(search_parameters)

            if solution:
                total_time = print_solution(manager, routing, solution)
                evaluation.append([i, j, total_time])
            else:
                #print("Solver status: ", routing.status())
                #print('No solution found !')
                evaluation.append([i, j, routing.status()])

    with open(os.path.join(folder_results, data_file[:-16] + '-results.csv'), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(evaluation)

homberger_200_C1_2_1-time_matrix.csv
homberger_200_R1_2_1-time_matrix.csv
homberger_200_RC1_2_1-time_matrix.csv
solomon-100-c101-time_matrix.csv
solomon-100-r101-time_matrix.csv
solomon-100-rc101-time_matrix.csv
solomon-25-c101-time_matrix.csv
solomon-25-r101-time_matrix.csv
solomon-25-rc101-time_matrix.csv
solomon-50-c101-time_matrix.csv
solomon-50-r101-time_matrix.csv
solomon-50-rc101-time_matrix.csv
